<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

In [1]:
import numpy as np
import scipy

from pprint import pprint
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation
#from scipy.linalg import expm

import time
import pickle
from pasqal_cloud import SDK
from pasqal_cloud.device import EmulatorType

import pulser
import qutip
from pulser import Pulse, Sequence, Register
from pulser_simulation import QutipEmulator
from pulser.waveforms import RampWaveform, ConstantWaveform
from pulser.devices import Chadoq2, MockDevice

/Users/Brian/workspace/Research/CO2_Capture_GNN/.conda/lib/python3.10/site-packages/qutip/__init__.py:65: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


## Cloud setup

In [2]:

project_id = "1d856b2d-58b1-4602-ad8a-e18f2db96bc3"
username = "br.ventura@reply.com"
password = "Ventur@007?"


CLOUD = False

## Graph MOF extraction

In [3]:
def MOF_reader(mol_str):
   with open('dataset_txt/'+mol_str+'_txt') as file:
    lines = file.readlines()
    N = len(lines)-1
    d = float(lines[0])
    plane_coords = np.array([lines[i].split() for i in range(1,N+1)], float)
    
    return d, plane_coords

In [4]:
MOF = "RURPAW"


In [5]:
# Parameters
MOF = "AHOKIR01"


In [6]:

d,coords = MOF_reader(MOF)
coords

array([[ 1.87750895, -2.11188591],
       [ 2.47459113, -3.05173109],
       [ 2.43510197, -2.43395445],
       [ 2.39487777, -3.65518912],
       [ 2.34807076, -1.72297621],
       [ 3.10969872, -3.03732382],
       [ 3.09868125, -2.70029021],
       [ 3.75353663, -2.99432677],
       [ 3.74202942, -2.39163576],
       [ 1.86541634, -3.17889212],
       [ 3.10990427, -3.63926526],
       [ 4.38949377, -3.00074944],
       [ 5.00070906, -3.15505706],
       [ 5.62536661, -3.21876504],
       [ 5.84214647, -3.7859023 ],
       [ 5.7792897 , -2.63158946],
       [ 6.23473864, -3.18573904],
       [ 5.12492545, -3.74291849],
       [ 5.0723226 , -2.76150168],
       [ 4.42454139, -3.59897056],
       [ 4.45670542, -2.4009102 ],
       [ 3.75878872, -3.34183315],
       [ 1.23431953, -2.11947686],
       [ 1.40052414, -2.73787125],
       [ 1.76229178, -2.50801099],
       [ 1.65665119, -1.54206996],
       [ 2.59535778, -4.21720424],
       [ 1.9031551 , -3.99323233],
       [ 2.12288962,

In [7]:
Omega = 1   * 2*np.pi
delta = 0.7 * 2*np.pi
U = 138e5

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:

n = len(coords)
#n = 6

def graph_desc(rNN):
   
    G1 = [rNN*np.array([c[0],c[1]]) for c in coords/d/3]
    #G1 = [[rNN*i, 0] for i in range(6)]
    return G1

G1 = graph_desc(15)


        
def excitation(j, total_sites):
    """The |0><0| projector operator on site j."""
    prod = [qutip.qeye(2) for _ in range(total_sites)]
    prod[j] = (qutip.qeye(2) + qutip.sigmaz()) / 2
    return qutip.tensor(prod)


def excitation2(l, n = 6):
    """The |0><0| projector operator on site j."""
    prod = [qutip.qeye(2) for _ in range(n)]
    for j in l:
        prod[j] = (qutip.qeye(2) + qutip.sigmaz()) / 2
    return qutip.tensor(prod)



excited = excitation2([0,1,2,3,4,5],n)

excited = excitation(5,n)


qubits1 = dict(enumerate(G1))
reg1 = Register(qubits1)
reg1.draw(blockade_radius=(U/Omega)**(1/6), draw_half_radius=True, draw_graph=True)

OverflowError: value too large

In [ ]:
seq = Sequence(reg1, MockDevice)
seq.declare_channel("ising", "rydberg_global")


M = 10000
sampling_rate = 1/100


In [ ]:
print('Enter your name:')
x

In [ ]:
K = 100
rNN_vec = np.linspace(25,3.,K)


In [ ]:
if(CLOUD):
    print("You are about to launch jobs on the cloud, continue ?")
    x = input()
    if(x=="y"):
        print('Good')
    else:
        raise SystemExit(0)


In [ ]:
#papermill_description=TQDM_DESCRIPTION

datafolder_Time_state_Evolution = "./dataset_txt/Time_evolution/"
datafolder_FFT_obs_Evolution = "./dataset_txt/FFT_feature/"



L = int(sampling_rate*M)
T = np.linspace(0,int(M/1000),L)
#STATE = np.zeros((K,L,2**n), dtype=complex)
STATE = []


results = []
for k in range(K):
    if(k%10==0):
        print(k)
    r = rNN_vec[k]
    G1 = graph_desc(r)
    
    qubits1 = dict(enumerate(G1))
    reg1 = Register(qubits1)

    seq = Sequence(reg1, MockDevice)
    seq.declare_channel("rydberg", "rydberg_global")
    

    
    simple_pulse = Pulse.ConstantPulse(M, Omega, 3*Omega, 0)
    
    #_omega = ConstantWaveform(M, Omega/10)
    #_omega = RampWaveform(M, rNN_vec[0]**6/1000, rNN_vec[24]**6/1000)
    #_delta = RampWaveform(M, 4, 20)  # Duration: 1000ns, linear sweep from -20 to 20 rad/µs
    #_delta = ConstantWaveform(M, 0.7*2*np.pi)
    
    #simple_pulse = Pulse(_omega,_delta,phase=0)

    seq.add(simple_pulse, "rydberg")
    seq.measure(basis="ground-rydberg")
    
    serialized_sequence = seq.to_abstract_repr()

    
    if(not CLOUD):
        sim = QutipEmulator.from_sequence(seq,sampling_rate=sampling_rate)
        result = sim.run(progress_bar=False, nsteps=500000)
        # Store state vector
        #for i in range(L):
        #    STATE[k,i,:] = np.array(result.get_state(T[i])).reshape(2**n)
        #
        STATE.append(result)
        results.append(result.expect([excited])[0])
    else:
        # Initialize the cloud client
        sdk = SDK(username=username, password=password, project_id=project_id)
        # When creating a job, select a number of runs and set the desired values for the variables
        # defined in the sequence
        job1 = {"runs": 1000}

        # You can also choose to run your batch on an emulator using the optional argument 'emulator'
        # For using a basic single-threaded QPU emulator that can go up to 10 qubits, you can specify the "EMU_FREE" emulator.
        batch = sdk.create_batch(
            serialized_sequence, [job1], emulator=EmulatorType.EMU_TN, wait=True
             )

            # print(f"job-id: {job.id}, status: {job.status}, result: {job.result}")
        #for job in batch.ordered_jobs:
            #print(f"job-id: {job.id}, status: {job.status}, result: {job.result}")
        #job.result

qutip.qsave(STATE, datafolder_Time_state_Evolution+MOF)

qutip.qsave(results, datafolder_FFT_obs_Evolution+MOF)
    #results.append(result.expect([excited])[0])


In [ ]:
#STATE = qutip.qload(datafolder_Time_state_Evolution+MOF)


In [ ]:
#

In [ ]:
results = np.array(results)

#T = np.linspace(0,4,int(M*sampling_rate))

fig, ax = plt.subplots()
l, = plt.plot(T,results[0,:])
ax.set_ylabel('y')
ax.set_xlabel('x')
#ax.set_ylim([0,2])

def animate(i):
    l.set_data(T,results[i,:])
    ax.set_title(rNN_vec[i])
ani = matplotlib.animation.FuncAnimation(fig, animate, interval=25, frames=K)

       
HTML(ani.to_jshtml())

In [ ]:
#print(dt)
#print((scipy.fft.fftfreq(M*10, dt/10)))
#print(results)
#print(1/dt)
#print(scipy.fft.fftfreq(M, dt))
int(M*sampling_rate)

In [ ]:
# For better frequency resolution



FFT = np.zeros((K,int(M*sampling_rate//2)))
FFTw = np.zeros((K,int(M*sampling_rate//2)))
dt = T[1]-T[0]
xf = scipy.fft.fftfreq(int(M*sampling_rate), dt)[:int(M*sampling_rate//2)]

from scipy.signal.windows import blackman
#window = blackman(T.shape[-1]//samplinsg_rate+1)

for i in range(K):
    results_sampled = results[i,:]

    FFT[i,:] = 2/M/sampling_rate*np.abs(scipy.fft.fft(results_sampled)[:int(M*sampling_rate//2)])
    #FFTw[i,:] = 2/M*np.abs(scipy.fft.fft(results_sampled*window)[:M//sampling_rate//2])
    


In [ ]:
print(len(xf))
print(xf)


In [ ]:
fig, ax = plt.subplots()
l, = plt.plot(xf,FFT[0,:])
ax.set_ylabel('y')
ax.set_xlabel('x')
#ax.set_ylim([0,0.75])
ax.set_xlim([0,5])

def animate(i):
    l.set_data(xf,FFT[i,:])
    ax.set_title(rNN_vec[i])
ani = matplotlib.animation.FuncAnimation(fig, animate, interval=100, frames=K)

       
HTML(ani.to_jshtml())

# ani.save('FFT_rNN_hexa.gif')

In [ ]:
np.log(FFT[0,:])

In [ ]:
print(Omega/2/np.pi*np.sqrt(n))

In [ ]:

plt.imshow(results, cmap='plasma', interpolation='nearest',extent=(0,4,25,3), aspect='auto')
plt.show()

In [ ]:
#fftbis = np.array([FFT[i,:],rNN_vec[i] for i in range(100)])
#print(fftbis.shape)
#fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,6))
plt.rcParams['text.usetex'] = False
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(15,6))
ax1.set_ylabel(r'$r_{NN}\, [\mu m]$',rotation=0,labelpad=20)#, loc='top')
ax1.set_xlabel(r'$\nu\,[s^{-1}]$',rotation=0,labelpad=10)#, loc='right')
im = ax1.imshow(np.log(FFT[:,:]),cmap='plasma',origin='lower', interpolation='nearest', extent=(0,4,25,3), aspect='auto')
#ax2.imshow(np.log(FFTw),cmap='plasma',origin='lower', interpolation='nearest', extent=(0,4,25,0.5))
#ax1.setgrid()
ax1.set_title("")

import matplotlib.ticker as ticker
def fmt(x, pos):
    #a, b = '{:.2e}'.format(x).split('e')
    #b = int(b)
    return r'$10^{{{}}}$'.format(int(x))

plt.colorbar(im, format=ticker.FuncFormatter(fmt))
#fig.colorbar(im)
#ax1.grid(alpha=0.5, linestyle='-', linewidth=1)
G1 = graph_desc(11)

qubits1 = dict(enumerate(G1))
reg1 = Register(qubits1)
reg1.draw(blockade_radius=(U/Omega)**(1/6), draw_half_radius=True, draw_graph=True,custom_ax=ax2)


plt.show()